In [ ]:
import torch  # see https://pytorch.org/tutorials/beginner/examples_nn/two_layer_net_nn.html
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import sklearn as skl
from sklearn import model_selection

In [ ]:
nskip = 1000  # Use only every 1000th data point
skip = np.arange(1e6)
skip = np.delete(skip, np.arange(0, 1e6, nskip))

data = pd.read_table(
    '/home/calbert/Dropbox/ipp/paper_algae/mc_out.dat',
    sep='\s+', skiprows=skip)
names = {
    'mu_0': 'k_alg_growth_max',
    'f_si': 'frac_si_alg_1',
    'lambda_S': 'k_att_shade',
    'K_light': 'k_light_sm',
    'sigma_0': 'k_alg_loss',
    'a': 'coeff_d_loss_2'
}
indata = data[['k_alg_loss', 'frac_si_alg_1']].values
indata = (indata - np.min(indata,0))/np.max(indata - np.min(indata,0), 0)
train, test = model_selection.train_test_split(indata)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(2, 8), torch.nn.Tanh(),
    #torch.nn.Linear(8, 8), torch.nn.Tanh(),
    torch.nn.Linear(8, 1), torch.nn.Tanh(),
    torch.nn.Linear(1, 8), torch.nn.Tanh(),
    #torch.nn.Linear(8, 8), torch.nn.Tanh(),
    torch.nn.Linear(8, 2)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

In [ ]:
train_torch = torch.from_numpy(train).to(torch.float32)
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
for t in range(10000):
    def closure():
        optimizer.zero_grad()
        output = model(train_torch)
        loss = loss_fn(output, train_torch)
        if t % 1000 == 99:
            print(t, loss.item())
        loss.backward()
        return loss

    optimizer.step(closure)

In [ ]:
with torch.no_grad():
    output_eval_train = model(train_torch)
    fig = px.scatter(x=train[:,0], y=train[:,1], labels={'x': 'sigma_0', 'y': 'f_si'}, title='Training data')
    fig.add_scatter(x=output_eval_train[:,0], y=output_eval_train[:,1], mode='markers', name='autoencoder')
    fig.show()
